In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from numpy import nan

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('max_seq_item', None)

In [52]:
# DATA SOURCE 1: DATASET FROM INFORMATION AND CONDITION OF SCHOOLS (ICOS), Pre-Disaster Mitigation module.
# Received from WA Office of Superintendent of Public Instruction (OSPI), 8/14/24, via public records request.
# https://www.k12.wa.us/policy-funding/school-buildings-facilities/information-and-condition-schools-icos
df_icos_original=pd.read_csv("data/df_icos_clean_8.14.24.csv")

In [53]:
# Copy the data and work with the copy.
df_icos = df_icos_original

In [54]:
df_icos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6678 entries, 0 to 6677
Data columns (total 32 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Unique_Identifier                    6678 non-null   object 
 1   District                             6678 non-null   object 
 2   Site                                 6678 non-null   object 
 3   Site_Latitude                        6678 non-null   float64
 4   Site_Longitude                       6678 non-null   float64
 5   Site_Type                            6678 non-null   object 
 6   Site_EQHazardLevel                   6678 non-null   object 
 7   Site_EQRiskLevel                     6678 non-null   object 
 8   Site_Soil                            6638 non-null   object 
 9   Site_Liquefaction                    6638 non-null   object 
 10  Site_GroundMotionPercentile          6638 non-null   float64
 11  Site_HasGeotechnicalStudy     

In [55]:
# converts values to a numeric data type
df_icos['BuildingID'] = df_icos['BuildingID'].astype('Int64')
df_icos['Building_YearBuilt'] = df_icos['Building_YearBuilt'].astype('Int64')

In [56]:
# How many seismic evaluations conducted in the School Seismic Safety Project are not shown in ICOS?

In [57]:
# Gets rid of extra spaces that could prevent merging
df_icos['Site'] = df_icos['Site'].str.strip()
df_icos['Building'] = df_icos['Building'].str.strip()

In [59]:
# DATA SOURCE 2: SCHOOL SEISMIC SAFETY PROJECT (SSSP), Washington State Department of Natural Resources.
# SSSP Phase 2 Report, 2019-2021 Legislative Report (6/30/21), table on pages 94-141. Data scraped from:
# https://fortress.wa.gov/dnr/geologydata/school_seismic_safety/phase2/School_Seismic_Safety_Project_2021_Final_Report_DNR.pdf
df_sssp_original=pd.read_csv("data/sssp_2021_legislative_report.csv")

In [60]:
# Copy the data and work with the copy
df_sssp = df_sssp_original

In [61]:
# Rename columns to match ICOS columns for merging. The "ICOS#" column in SSSP refers to the BuildingID column 
# in ICOS (source: OSPI). 
df_sssp.rename(columns={'District Name': 'District', 'Facility Name': 'Site','Building Name': 'Building', \
                                     'ICOS#': 'BuildingID'}, inplace=True)

In [62]:
# converts values to a numeric data type
df_sssp['BuildingID'] = df_sssp['BuildingID'].astype('Int64')

In [63]:
# These additional columns will be needed later to make edits for matching with ICOS values.
# The "original" columns will retain original values, and edits will be made to the columns on the right.
df_sssp['original_BuildingID_SSSP'] = df_sssp['BuildingID']
df_sssp['original_Building_SSSP'] = df_sssp['Building']

In [64]:
# According to the SSSP, the project assessed 561 school buildings (274 schools at 245 campuses).
# There are 516 Building IDs in the SSSP data. 
sssp_buildings = df_sssp['BuildingID'].nunique()
print("There are", sssp_buildings, "Building IDs in the SSSP data.")

There are 516 Building IDs in the SSSP data.


In [65]:
# Analysis further below will show that some records with matching Building IDs in both data sources do
# not match by school or building name. So matching by Building ID alone could lead to errors.
# School and building names are also needed for matching records between the two data sources.

In [66]:
# DATA CLEANING
# Three Building IDs from the SSSP are missing in ICOS, because ICOS has different IDs for two buildings, 
# and one school closed. In addition, one school name is misspelled in the SSSP data. 
# Below, these SSSP data discrepancies are fixed to match the data in ICOS.

In [67]:
# removes extra spaces that can prevent merging
df_sssp['Site'] = df_sssp['Site'].str.strip()

In [68]:
# Create a one-column list of the 516 unique Building IDs in the SSSP to search ICOS for BuildingID matches.
df_sssp2 = df_sssp['BuildingID']
df_sssp2 = df_sssp2.drop_duplicates()

# Columns became a list of BuildingIDs; reset the index to restore the dataframe to two columns.
df_sssp2 = df_sssp2.reset_index() 

# Capture only the BuildingID column. This variable can be used to search ICOS.
df_sssp_ids = df_sssp2['BuildingID']

In [69]:
# Find all rows in ICOS with Building IDs that are also found in the SSSP. 
df_icos2 = df_icos[df_icos['BuildingID'].isin(df_sssp_ids)]
df_icos2.shape

(865, 32)

In [70]:
# There are 516 unique building IDs in the SSSP, but only 513 of these are found in ICOS. 
df_icos_ids = df_icos2['BuildingID'].drop_duplicates()
df_icos_ids.shape

(513,)

In [71]:
# Identify the three building IDs from the SSSP that do not appear in ICOS.
df_bldgs_missing = df_sssp_ids[~df_sssp_ids.isin(df_icos_ids)]
df_bldgs_missing

43     50960
414    53918
416    53919
Name: BuildingID, dtype: Int64

In [72]:
# These are the three buildings with Building IDs from the SSSP that are not in ICOS. 
df_bldgs_missing2 = df_sssp[df_sssp['BuildingID'].isin(df_bldgs_missing)]
df_bldgs_missing2

,Priority level,District,Site,Building,BuildingID,original_BuildingID_SSSP,original_Building_SSSP
43,Very High,Mount Vernon,Lincoln Elementary School,Main Building,50960,50960,Main Building
446,Moderate,Yakima,Robertson Elementary School,"100 Building - Bldg ""B""",53918,53918,"100 Building - Bldg ""B"""
448,Moderate,Yakima,Robertson Elementary School,"300 Building - Bldg ""D""",53919,53919,"300 Building - Bldg ""D"""


In [73]:
# Lincoln Elementary School in Mount Vernon is not in ICOS, and online sources say it closed in 2021,
# although the building was still being used by students from another school as recently as 2021. 
# https://www.publicschoolreview.com/lincoln-elementary-school-profile/98274
df_icos_bldg_search = df_icos[['District', 'Site', 'BuildingID','Building', 'BuildingArea_Name']]

mask = (df_icos_bldg_search['Site'] == 'Lincoln Elementary School') & \
(df_icos_bldg_search['District'] == 'Mount Vernon')

df_bldgs_missing3 = df_icos_bldg_search[mask]
df_bldgs_missing3.shape

(0, 5)

In [74]:
# The other two buildings are in ICOS with different Building IDs than in the SSSP.
mask = (df_icos_bldg_search['Site'] == 'Robertson Elementary School') & \
(df_icos_bldg_search['District'] == 'Yakima') & \
(df_icos_bldg_search['Building'].str.contains('100 Building'))

mask2 = (df_icos_bldg_search['Site'] == 'Robertson Elementary School') & \
(df_icos_bldg_search['District'] == 'Yakima') & \
(df_icos_bldg_search['Building'].str.contains('300 Building'))

df_bldgs_missing4 = df_icos_bldg_search[mask]
df_bldgs_missing5 = df_icos_bldg_search[mask2]
df_bldgs_missing6 = (df_bldgs_missing4, df_bldgs_missing5)
df_bldgs_missing7 = pd.concat(df_bldgs_missing6)
df_bldgs_missing7

,District,Site,BuildingID,Building,BuildingArea_Name
6614,Yakima,Robertson Elementary School,53928,100 Building - B,Main Area
6616,Yakima,Robertson Elementary School,53922,300 Building - D,Main Area


In [75]:
# This school is misspelled in the SSSP. The correct spelling is "Prairie": https://phs.battlegroundps.org/
mask = (df_sssp['Site'] == 'Praire High School') 
df_sssp_bldgs_typo = df_sssp[mask]
df_sssp_bldgs_typo

,Priority level,District,Site,Building,BuildingID,original_BuildingID_SSSP,original_Building_SSSP
87,High,Battle Ground,Praire High School,500 Building,50021,50021,500 Building
88,High,Battle Ground,Praire High School,600 Building,50024,50024,600 Building
459,Lower,Battle Ground,Praire High School,400 Building,50013,50013,400 Building


In [76]:
# This is the correct spelling of the school name.
df_icos_bldg_search2 = df_icos_bldg_search[['District', 'Site']]

mask = (df_icos_bldg_search2['Site'] == 'Prairie High School') 

df_sssp_bldgs_typo3 = df_icos_bldg_search2[mask]
df_sssp_bldgs_typo4 = df_sssp_bldgs_typo3.drop_duplicates()
df_sssp_bldgs_typo4

,District,Site
225,Battle Ground,Prairie High School


In [77]:
# Copy the SSSP data and adjust the two Building IDs that did not match ICOS so that they will match, 
# and fix the school name typo in the SSSP so the school name will match ICOS.
df_sssp3 = df_sssp

In [78]:
df_sssp3.at[446, 'BuildingID'] = 53928
df_sssp3.at[448, 'BuildingID'] = 53922
df_sssp3.at[87, 'Site'] = 'Prairie High School'
df_sssp3.at[88, 'Site'] = 'Prairie High School'
df_sssp3.at[459, 'Site'] = 'Prairie High School'

In [79]:
# Verify that the typo in this school name has been fixed in the SSSP data.
df_sssp3.iloc[[87, 88, 459]]

,Priority level,District,Site,Building,BuildingID,original_BuildingID_SSSP,original_Building_SSSP
87,High,Battle Ground,Prairie High School,500 Building,50021,50021,500 Building
88,High,Battle Ground,Prairie High School,600 Building,50024,50024,600 Building
459,Lower,Battle Ground,Prairie High School,400 Building,50013,50013,400 Building


In [80]:
df_sssp3.iloc[[446, 448]]

,Priority level,District,Site,Building,BuildingID,original_BuildingID_SSSP,original_Building_SSSP
446,Moderate,Yakima,Robertson Elementary School,"100 Building - Bldg ""B""",53928,53918,"100 Building - Bldg ""B"""
448,Moderate,Yakima,Robertson Elementary School,"300 Building - Bldg ""D""",53922,53919,"300 Building - Bldg ""D"""


In [81]:
df_sssp3.shape

(556, 7)

In [82]:
# Repeat several steps from above with the adjusted data to create a new list of SSSP building IDs to search ICOS.

In [83]:
df_sssp4 = df_sssp3['BuildingID']
df_sssp5 = df_sssp4.drop_duplicates()
df_sssp5.shape

(516,)

In [84]:
df_sssp5 = df_sssp5.reset_index()

In [85]:
# This single column list can now be used to search through the ICOS Building ID column.
df_sssp5 = df_sssp5['BuildingID']

In [86]:
df_icos_2 = df_icos[df_icos['BuildingID'].isin(df_sssp5)]
df_icos_2.shape

(867, 32)

In [87]:
# Identify the buildings in the SSSP that are marked as retrofitted in ICOS.
df_icos_2['BuildingArea_HasRetrofit'].value_counts()

BuildingArea_HasRetrofit
No            852
Structural     15
Name: count, dtype: int64

In [88]:
mask = (df_icos_2['BuildingArea_HasRetrofit'] == 'Structural')
df_icos_2_retrofit = df_icos_2[mask]
df_icos_2_retrofit[['District', 'Site', 'Building', 'BuildingArea_HasRetrofit']]\
   .drop_duplicates() # These buildings are marked as retrofited in the SSSP list, but there is no retrofit
# date to know if those retrofits happened before or after the project ended.

,District,Site,Building,BuildingArea_HasRetrofit
10,Aberdeen,McDermoth Elementary School,Main Building,Structural
43,Anacortes,Mount Erie Elementary School,Main Building,Structural
643,Carbonado,Carbonado Historical School 19,A - Main Building,Structural
1531,Everett,Jackson Elementary School,Main Building,Structural
3954,Oak Harbor,Oak Harbor Intermediate School,Building B,Structural
3955,Oak Harbor,Oak Harbor Intermediate School,C Wing,Structural
3957,Oak Harbor,Oak Harbor Intermediate School,D Wing,Structural
3959,Oak Harbor,Oak Harbor Intermediate School,Main Building A,Structural
6293,Wahkiakum,Julius A. Wendt Elementary/John C. Thomas Middle School,J A Wendt Elementary School,Structural


In [89]:
# There are 867 rows in ICOS that match 515 Building IDs in the SSSP. 
# The adjustments above correctly raised the totals by 2 each.
df_icos_2['BuildingID'].nunique()

515

In [90]:
# DATA CLEANING: Identify school names that do not match between the SSSP and ICOS. See if small adjustments can be 
# made to school names in the SSSP to match ICOS for merging when they clearly refer to the same school.

In [91]:
# Create a unique ID combining school and district because some school names are used by multiple districts.
df_sssp3['Unique_Identifier'] = df_sssp3['District'].astype(str) +"_"+ df_sssp3['Site'].astype(str)

In [92]:
df_sssp3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Priority level            556 non-null    object
 1   District                  556 non-null    object
 2   Site                      556 non-null    object
 3   Building                  556 non-null    object
 4   BuildingID                556 non-null    Int64 
 5   original_BuildingID_SSSP  556 non-null    Int64 
 6   original_Building_SSSP    556 non-null    object
 7   Unique_Identifier         556 non-null    object
dtypes: Int64(2), object(6)
memory usage: 36.0+ KB


In [93]:
df_sssp_sites = df_sssp3[['Unique_Identifier', 'District', 'Site']].drop_duplicates()
df_sssp_sites.shape
# The SSSP reports say there were 274 schools at 245 campuses in the study. This shows 277. This could be due
# to name variations making this appear to be several more schools than the actual number.

(277, 3)

In [94]:
df_icos_sites = df_icos_2[['Unique_Identifier', 'District', 'Site']].drop_duplicates()
df_icos_sites.shape

(271, 3)

In [95]:
df_sssp_ids = df_sssp_sites['Unique_Identifier'].drop_duplicates()
df_sssp_ids.shape

(277,)

In [96]:
df_icos_ids = df_icos_sites['Unique_Identifier'].drop_duplicates()
df_icos_ids.shape
# There are 271 schools in ICOS that have matched the SSSP school names so far.

(271,)

In [97]:
df_test1 = df_icos_sites[~df_icos_sites['Unique_Identifier'].isin(df_sssp_ids)]
df_test1.shape

(15, 3)

In [98]:
df_test1['Unique_Identifier'].unique() # These are the schools in ICOS not found in the SSSP, despite
# having common Building IDs

array(['Bellingham_Happy Valley Elementary School ',
       'Burlington-Edison_West View Elementary School',
       'Cape Flattery_Clallam Bay K-12 School',
       'Cape Flattery_Neah Bay Secondary School',
       'Central Kitsap_Ridgetop Middle School',
       'La Conner_La Conner Middle School (form. Elem site.)',
       'Mount Baker_Mount Baker Junior High School',
       'Oak Harbor_Clover Valley Home Connection',
       'Oak Harbor_Oak Harbor Intermediate School',
       'Quillayute Valley_Forks Middle School',
       'South Whidbey_South Whidbey South Campus - (Formerly S. Whid. Primary)',
       'Spokane_Bryant',
       'Wahkiakum_Julius A. Wendt Elementary/John C. Thomas Middle School',
       'Warden_Middle / High School',
       'Woodland_Woodland Middle School (old HS)'], dtype=object)

In [99]:
df_test2 = df_sssp_sites[~df_sssp_sites['Unique_Identifier'].isin(df_icos_ids)]
df_test2.shape

(21, 3)

In [100]:
df_test2['Unique_Identifier'].unique() # These are the schools in the SSSP not found in the ICOS list that was
# pulled in via common Building IDs. Adjust names with minor variations to match the ICOS list above.
# Any schools listed here and not in the ICOS list above may have been missed due to Building ID differences.

array(['Burlington-Edison_West View Elementary School School',
       'Cape Flattery_Clallam Bay High and Elementary School',
       'Cape Flattery_Neah Bay Junior/ Senior High  School',
       'La Conner_La Conner Middle School (form. Elem.)',
       'Marysville_Marysville Pilchuck Sr High School',
       'Mount Vernon_Lincoln Elementary School',
       'Quillayute Valley_Forks Intermediate School',
       'Quillayute Valley_Forks Jr-Sr High School',
       'Woodland_Woodland Middle School',
       'Cape Flattery_Neah Bay Jr/ Sr High School',
       'Oak Harbor_Clover Valley School',
       'Oak Harbor_Oak Harbor Middle School',
       'South Whidbey_South Whidbey Grades 5 & 6',
       'Spokane_Bryant Center',
       'Wahkiakum_Julius A. Wendt ES/John C. Thomas MS',
       'Cape Flattery_Neah Bay Junior/ Senior High School',
       'Central Kitsap_Ridgetop Junior High',
       'Mount Baker_Mount Baker Jr High School',
       'Mount Baker_Mount Baker Sr High School',
       'Thorp_Thor

In [101]:
df_sssp3['Site'] = df_sssp3['Site'].replace(["West View Elementary School School"], "West View Elementary School)")
df_sssp3['Site'] = df_sssp3['Site'].replace(["La Conner Middle School (form. Elem.)"], \
                                            "La Conner_La Conner Middle School (form. Elem site.)")
df_sssp3['Site'] = df_sssp3['Site'].replace(["Mount Baker_Mount Baker Jr High School"], \
                                            "Mount Baker Junior High School)")
df_sssp3['Site'] = df_sssp3['Site'].replace(["Clover Valley School"], \
                                            "Clover Valley Home Connection")
df_sssp3['Site'] = df_sssp3['Site'].replace(["Oak Harbor Middle School"], \
                                            "Oak Harbor Intermediate School")
df_sssp3['Site'] = df_sssp3['Site'].replace(["Forks Jr-Sr High School"], \
                                            "Forks Junior-Senior High School")
df_sssp3['Site'] = df_sssp3['Site'].replace(["South Whidbey_South Whidbey Grades 5 & 6"], \
                                            "South Whidbey South Campus - (Formerly S. Whid. Primary)'")
df_sssp3['Site'] = df_sssp3['Site'].replace(["Bryant Center"], "Bryant")
df_sssp3['Site'] = df_sssp3['Site'].replace(["Julius A. Wendt ES/John C. Thomas MS"], \
                                            "Julius A. Wendt Elementary/John C. Thomas Middle School")
df_sssp3['Site'] = df_sssp3['Site'].replace(["Ridgetop Junior High"], \
                                            "Ridgetop Middle School")
df_sssp3['Site'] = df_sssp3['Site'].replace(["Warden_Warden K-12"], \
                                            "Warden_Middle / High School")
df_sssp3['Site'] = df_sssp3['Site'].replace(["Woodland Middle School"], \
                                            "Woodland Middle School (old HS)")

In [102]:
# Test to see which ones still do not match now.

In [103]:
df_sssp_sites2 = df_sssp3[['District', 'Site']].drop_duplicates()
df_sssp_sites2.shape

(277, 2)

In [104]:
# Make unique identifiers again with the adjusted names
df_sssp_sites2['Unique_Identifier'] = df_sssp_sites2['District'].astype(str) +"_"+ \
     df_sssp_sites2['Site'].astype(str)

In [105]:
df_sssp_sites3 = df_sssp_sites2[['Unique_Identifier', 'District', 'Site']].drop_duplicates()
df_sssp_sites3.shape

(277, 3)

In [106]:
df_sssp_ids2 = df_sssp_sites2['Unique_Identifier'].drop_duplicates()
df_sssp_ids2.shape

(277,)

In [107]:
df_test3 = df_icos_sites[~df_icos_sites['Unique_Identifier'].isin(df_sssp_ids2)]
df_test3.shape
# There are still 9 schools that were not pulled in via Building IDs.

(9, 3)

In [108]:
# MERGING

In [109]:
# Make copies for merging.
df_icos_to_combine = df_icos_2
df_sssp_to_combine = df_sssp3

In [110]:
df_sssp_bldgs1 = df_sssp3[['District', 'Site', 'Building']].drop_duplicates()
df_sssp_bldgs1.shape

(556, 3)

In [111]:
# There are 516 unique building IDs for the 556 building areas in the SSSP data. These match to 515 IDs
# in ICOS, since one school closed and is not in ICOS. The ICOS rows for these buildings total 867.

In [112]:
# MERGE
# df_icos_to_combine      (867 rows, 515 Building IDs, 32 columns)
# df_sssp_to_combine      (556 rows, 516 Building IDs, 6 columns)

In [113]:
# Adjust building names for matching where slight, non-substantive differences are present. 
# Some building IDs are used with multiple building area rows.

In [114]:
df_bldg_names_icos = df_icos_to_combine[['Site','BuildingID', 'Building', 'BuildingArea_Name']]
# This contains 515 building IDs, each used once.

In [115]:
df_bldg_names_icos.shape

(867, 4)

In [116]:
df_bldg_names_icos2 = df_bldg_names_icos.drop_duplicates()
df_bldg_names_icos2.shape
# This lists has one row per Building ID, 515 without building area name added above. 

(807, 4)

In [117]:
df_bldg_names_icos2['BuildingID'].nunique()

515

In [118]:
df_bldg_names_icos2['Site'].nunique() # there are 262 schools in the ICOS list of 515 buildings

262

In [119]:
df_bldg_names_sssp = df_sssp_to_combine[['Site','BuildingID', 'Building']]

In [120]:
df_bldg_names_sssp.shape

(556, 3)

In [121]:
merged_sssp_icos_bldgs = df_bldg_names_sssp.merge(df_bldg_names_icos2, how = 'left', on = ['BuildingID'])

In [122]:
merged_sssp_icos_bldgs.shape

(894, 6)

In [123]:
df_icos_to_combine_2 = df_icos_to_combine.groupby(['District', 'Site', 'BuildingID', 'Building', \
                'Building_HasEvaluation', 'Building_YearBuilt'], \
                as_index=False).agg({'BuildingArea_ExistingRisk': ' '.join})

In [124]:
merged_sssp_icos_df = df_sssp_to_combine.merge(df_icos_to_combine_2, how = 'inner', on = ['BuildingID', \
                                                                                          'District'])

In [125]:
merged_sssp_icos_df.shape
# There are 555 building names in the SSSP, matched to 515 building IDs in ICOS, some of which are repeated.

(555, 13)

In [126]:
merged_sssp_icos_df['building match'] = merged_sssp_icos_df.apply(lambda x: 'Yes' if\
                                                             x['Building_x'] in x['Building_y'] else 'No',axis=1)

In [127]:
merged_sssp_icos_df['site match'] = merged_sssp_icos_df.apply(lambda x: 'Yes' if\
                                                             x['Site_x'] in x['Site_y'] else 'No',axis=1)

In [128]:
merged_sssp_icos_df.rename(columns={'Site_x': 'Site_SSSP', 'Site_y': 'Site_ICOS', 'Building_x': 'Building_SSSP', \
                                     'Building_y': 'Building_ICOS'}, inplace=True)

In [129]:
# Check the school names that did not match and see if the differences are minor or substantial.
merged_sssp_icos_df2 = merged_sssp_icos_df[['Site_SSSP', 'Site_ICOS', 'site match']]
mask = (merged_sssp_icos_df2['site match'] == 'No')
merged_sssp_icos_df3 = merged_sssp_icos_df2[mask]
merged_sssp_icos_df3 = merged_sssp_icos_df3.drop_duplicates()
merged_sssp_icos_df3
# By observation, these are all correctly matched with minor variations. One exception is "Warden K-12" in the 
# SSSP, which is listed as "Middle / High School" in ICOS, but the Warden district has only two schools in ICOS,  
# and the other one is an elementary school, so this appears correctly matched too. https://www.warden.wednet.edu/
# Use the SSSP site list in subsequent dataframes and ignore the ICOS site list.

,Site_SSSP,Site_ICOS,site match
11,West View Elementary School),West View Elementary School,No
12,Clallam Bay High and Elementary School,Clallam Bay K-12 School,No
14,Neah Bay Junior/ Senior High School,Neah Bay Secondary School,No
33,La Conner_La Conner Middle School (form. Elem site.),La Conner Middle School (form. Elem site.),No
37,Marysville Pilchuck Sr High School,Marysville Pilchuck Senior High School,No
55,Forks Intermediate School,Forks Middle School,No
56,Forks Junior-Senior High School,Forks Middle School,No
110,Neah Bay Jr/ Sr High School,Neah Bay Secondary School,No
310,South Whidbey Grades 5 & 6,South Whidbey South Campus - (Formerly S. Whid. Primary),No
463,Fairhaven Middle School,Happy Valley Elementary School,No


In [130]:
# Create a list of building names that do not match between the SSSP and ICOS. There are 115 mis-matched names.
merged_sssp_icos_df_buildings = merged_sssp_icos_df[['BuildingID', 'original_Building_SSSP', 'Building_SSSP', \
                  'Building_ICOS', 'building match', 'Building_HasEvaluation', 'BuildingArea_ExistingRisk']]
mask = (merged_sssp_icos_df_buildings['building match'] == 'No')
merged_sssp_icos_df_buildings2 = merged_sssp_icos_df_buildings[mask]
merged_sssp_icos_df_buildings2 = merged_sssp_icos_df_buildings2.drop_duplicates()
# drop buildings match column
merged_sssp_icos_df_buildings2.shape

(123, 7)

In [131]:
# Check the building names that did not match and see if the differences are minor or substantial.
# For small, non-substantive variations, make adjustments to SSSP building names to match ICOS.
# merged_sssp_icos_df_buildings2[['Building_SSSP', 'Building_ICOS']]

In [132]:
# Adjust minor variations in the SSSP building names to match ICOS. Keep the original names too.
# First, do this just to the list of buildings with mismatched names. Verify that it looks right.
# Then apply the changes to the whole merged dataframe of all buildings, including those previously matched.
# After correcting the building names, re-merge.
buildings = [4, 10, 55, 56, 58, 61, 62, 65, 69, 70, 71, 74, 75, 76, 157, 158, 185, 187, 209, 212, 216, 217, 220, \
            246, ]
for i in buildings:
    merged_sssp_icos_df['Building_SSSP'].iloc[i] = merged_sssp_icos_df\
    ['Building_ICOS'].iloc[i]

In [133]:
# Apply the changes to the whole merged dataframe
buildings = [1, 2, 4, 5]
for i in buildings:
    merged_sssp_icos_df['Building_SSSP'].iloc[i] = merged_sssp_icos_df\
    ['Building_ICOS'].iloc[i]

In [134]:
# Make a copy to work with
df_sssp_to_combine = df_sssp3

In [135]:
# Repeat merge with corrected data. 
merged_sssp_icos_df2 = df_sssp_to_combine.merge(df_icos_to_combine_2, how = 'inner', on = ['BuildingID', \
                                                                                          'District'])

In [136]:
merged_sssp_icos_df2['buildings match'] = merged_sssp_icos_df2.apply(lambda x: 'Yes' if\
                                                             x['Building_x'] in x['Building_y'] else 'No',axis=1)

In [137]:
merged_sssp_icos_df2['Building_HasEvaluation'].value_counts()

Building_HasEvaluation
Yes    347
No     208
Name: count, dtype: int64

In [138]:
merged_sssp_icos_df2.rename(columns={'Site_x': 'Site_SSSP', 'Site_y': 'Site_ICOS', 'Building_x': 'Building_SSSP', \
                                     'Building_y': 'Building_ICOS'}, inplace=True)

In [140]:
# Re-run the above code to check how many building names are still mis-matched, with substantial differences.
merged_sssp_icos_df2_buildings = merged_sssp_icos_df2[['Building_SSSP', 'Building_ICOS', 'buildings match', \
                                                      'Building_HasEvaluation']]
mask = (merged_sssp_icos_df2_buildings['buildings match'] == 'No')
merged_sssp_icos_df2_buildings2 = merged_sssp_icos_df2_buildings[mask]
merged_sssp_icos_df2_buildings3 = merged_sssp_icos_df2_buildings2.drop_duplicates()
# merged_sssp_icos_df2_buildings3

In [142]:
# Have any buildings been replaced since the SSSP data was collected?
# The SSSP started in 2017. Does the data show any buildings that were replaced since then, which
# could explain missing evaluations? 
mask = (merged_sssp_icos_df2['Building_YearBuilt'] > 2016)

merged_sssp_icos_df3 = merged_sssp_icos_df2[mask]
merged_sssp_icos_df3.shape
# Answer: no, there are none.

(0, 14)

In [143]:
# EVALUATIONS

In [144]:
merged_sssp_icos_df2['BuildingID'].nunique()

515

In [145]:
merged_sssp_icos_df2['Building_HasEvaluation'].value_counts()

Building_HasEvaluation
Yes    347
No     208
Name: count, dtype: int64

In [146]:
mask = (merged_sssp_icos_df2['Building_HasEvaluation'] == 'Yes')
merged_sssp_icos_eval = merged_sssp_icos_df2[mask]
merged_sssp_icos_eval.shape

(347, 14)

In [147]:
merged_sssp_icos_eval['BuildingID'].nunique() # there are 316 out of 516 buildings with one or more evaluation

330

In [148]:
mask = (merged_sssp_icos_df2['Building_HasEvaluation'] == 'No')
merged_sssp_icos_no_eval = merged_sssp_icos_df2[mask]
merged_sssp_icos_no_eval.shape

(208, 14)

In [149]:
merged_sssp_icos_no_eval['BuildingID'].nunique() # There are 185 buildings without an evaluation.
# Some buildings have more than one 'building area' row, yielding 208 rows for these 185 buildings.

185

In [150]:
mask = merged_sssp_icos_no_eval['buildings match'] == 'Yes'
merged_sssp_icos_no_eval_match = merged_sssp_icos_no_eval[mask]
merged_sssp_icos_no_eval_match['BuildingID'].nunique() 
# There are 160 buildings with matching names and matching building IDs between the SSSP and ICOS
# that have no evaluations in ICOS.

160

In [151]:
# Confirming that these buildings have no evaluations
merged_sssp_icos_no_eval_match['Building_HasEvaluation'].value_counts()

Building_HasEvaluation
No    160
Name: count, dtype: int64

In [153]:
merged_sssp_icos_no_eval_match[['District', 'Site_SSSP', 'Site_ICOS', 'original_Building_SSSP', \
                         'Building_ICOS', 'BuildingID', 'original_BuildingID_SSSP', 'Building_HasEvaluation']]

,District,Site_SSSP,Site_ICOS,original_Building_SSSP,Building_ICOS,BuildingID,original_BuildingID_SSSP,Building_HasEvaluation
7,Burlington-Edison,Burlington-Edison High School,Burlington-Edison High School,Art/Tiger TUB Building,Art/Tiger TUB Building,50119,50119,No
8,Burlington-Edison,Burlington-Edison High School,Burlington-Edison High School,Cafeteria and 400 Wing,Cafeteria and 400 Wing,50117,50117,No
9,Burlington-Edison,Burlington-Edison High School,Burlington-Edison High School,CTE,CTE,50110,50110,No
11,Burlington-Edison,West View Elementary School),West View Elementary School,Main Building,Main Building,50095,50095,No
17,Centralia,Washington Elementary School,Washington Elementary School,Main Building,Main Building,57962,57962,No
18,Clover Park,Tillicum Elementary School,Tillicum Elementary School,Classroom Building - TL1,Classroom Building - TL1,50186,50186,No
21,Ferndale,Custer Elementary,Custer Elementary,Main Building,Main Building,54976,54976,No
24,Hoquiam,Central Elementary School,Central Elementary School,Main Building,Main Building,58356,58356,No
25,Hoquiam,Emerson Elementary School,Emerson Elementary School,Main Building,Main Building,58357,58357,No
31,Kelso,Rose Valley Elementary School,Rose Valley Elementary School,Main Building,Main Building,58396,58396,No
